In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf

In [2]:
MRI = pd.read_excel("OASIS3_MR_json.xlsx")
PET = pd.read_excel("OASIS3_PET_json.xlsx")
CDR = pd.read_excel("OASIS3_UDSb4_cdr.xlsx")
APOE = pd.read_excel("demo.xlsx")

In [4]:
#Variables of interest
CDR_interest = CDR[["OASISID","OASIS_session_label","CDRTOT","days_to_visit","dx1"]].copy()
CDR_interest["OASIS_session_label"] = pd.to_numeric(CDR_interest["OASIS_session_label"].str.slice(16, 20))

#Sessions where subject has CDR 0
CDR_0_start = sqldf("select * from CDR_interest where CDRTOT = 0")


PET_PIB_required = PET[["subject_id","session_id"]].copy()
PET_PIB_required = PET_PIB_required[PET_PIB_required['session_id'].str.contains("_PIB_")]
PET_PIB_required["session_id"] = pd.to_numeric(PET_PIB_required["session_id"].str.slice(start=14))

MRI_required = MRI[["subject_id","label","scan category"]].copy()
MRI_required["label"] = pd.to_numeric(MRI_required["label"].str.slice(start=13))
MRI_required = sqldf("select DISTINCT subject_id, label from MRI_required where `scan category` == 'T1w' ")

In [8]:
#Those who converted within 10 years
CDR_pNC = sqldf("select CDR_0_start.OASISID, CDR_0_start.days_to_visit as start, CDR_interest.days_to_visit as end,\
CDR_interest.CDRTOT from CDR_0_start join CDR_interest on CDR_0_start.OASISID = CDR_interest.OASISID and\
(CDR_interest.days_to_visit - CDR_0_start.days_to_visit < 3650) and\
(CDR_interest.days_to_visit - CDR_0_start.days_to_visit > 0) and\
(CDR_interest.CDRTOT >=0.5)")

#Those who converted within 10 years and had a pet scan within a year from baseline
CDR_pNC_pib = sqldf("select CDR_pNC.OASISID, CDR_pNC.start, CDR_pNC.end, PET_PIB_required.session_id as PIB_scan \
from CDR_pNC join PET_PIB_required on CDR_pNC.OASISID = PET_PIB_required.subject_id and\
(PET_PIB_required.session_id - CDR_pNC.start <= 365) and\
(PET_PIB_required.session_id - CDR_pNC.start >= 0)")

#Those who converted within 10 years and had a pet+mri scan within a year from baseline
CDR_pNC_pib_mri = sqldf("select CDR_pNC_pib.OASISID, CDR_pNC_pib.start, CDR_pNC_pib.end, CDR_pNC_pib.PIB_scan,\
MRI_required.label as MRI_scan \
from CDR_pNC_pib join MRI_required on CDR_pNC_pib.OASISID = MRI_required.subject_id and\
(MRI_required.label - CDR_pNC_pib.start <= 365) and\
(MRI_required.label - CDR_pNC_pib.start >= 0)")

PNC =  sqldf("select distinct OASISID from CDR_pNC_pib_mri")



#Those who remained NC after 10 years
CDR_sNC = sqldf("select CDR_0_start.OASISID, CDR_0_start.days_to_visit as start, CDR_interest.days_to_visit as end, \
CDR_interest.CDRTOT from CDR_0_start join CDR_interest on CDR_0_start.OASISID = CDR_interest.OASISID and\
(CDR_interest.days_to_visit - CDR_0_start.days_to_visit >= 3650) and\
(CDR_interest.CDRTOT = 0)")

#Those who remained NC after 10 years and had a pet scan within a year from baseline
CDR_sNC_pib = sqldf("select CDR_sNC.OASISID, CDR_sNC.start, CDR_sNC.end, PET_PIB_required.session_id as PIB_scan \
from CDR_sNC join PET_PIB_required on CDR_sNC.OASISID = PET_PIB_required.subject_id and\
(PET_PIB_required.session_id - CDR_sNC.start <= 365) and\
(PET_PIB_required.session_id - CDR_sNC.start >= 0)")

#Those who remained NC after 10 years and had a pet+mri scan within a year from baseline
CDR_sNC_pib_mri = sqldf("select CDR_sNC_pib.OASISID, CDR_sNC_pib.start, CDR_sNC_pib.end, CDR_sNC_pib.PIB_scan, \
MRI_required.label as MRI_scan \
from CDR_sNC_pib join MRI_required on CDR_sNC_pib.OASISID = MRI_required.subject_id and\
(MRI_required.label - CDR_sNC_pib.start <= 365) and\
(MRI_required.label - CDR_sNC_pib.start >= 0) and CDR_sNC_pib.OASISID not in(select OASISID from PNC)")

SNC =  sqldf("select distinct OASISID from CDR_sNC_pib_mri where OASISID not in(select OASISID from PNC)")

In [18]:
PNC

,OASISID
0,OAS30007
1,OAS30025
2,OAS30027
3,OAS30028
4,OAS30038
...,...
102,OAS31128
103,OAS31141
104,OAS31162
105,OAS31164


In [12]:
SNC

,OASISID
0,OAS30001
1,OAS30005
2,OAS30006
3,OAS30009
4,OAS30015
...,...
98,OAS31125
99,OAS31132
100,OAS31157
101,OAS31159
